In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import yaml
import matplotlib.pyplot as plt


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_grids.txt', 'r') as file:   path_grids = file.read()
    
dir_data_Ymax  = f'{path_main}Data/CMIP6/Yearmax/'
dir_scripts    = f'{path_main}Scripts/'
dir_names      = f'{path_main}Scripts/Model_lists/'
dir_regrid     = f'{path_grids}Regridding/'


## Prepare variables and parameters

In [ ]:
#Define models
all_models = []
with open(dir_names + 'Models_CMIP6_SSP585.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(line[:-1])    
        
#Define reference period
ref_period = '1981-2010'

#Define EMT
dEMT_str = 'ESAT-3.0K'    

#Define limits for European box
lon_limits_EUR = [-10, 35]
lat_limits_EUR = [30, 70]


## Regrid TXx and TNx

In [ ]:
variables = ['TXx']#, 'TNx']

data_all = dict()

for variab in variables:

    dir_data = dir_data_Ymax + variab + '/'

    print(len(all_models), end=': ')

    #Loop over all models
    create = 1
    for i, model in enumerate(all_models):        
        
        print(i, end=', ')

        #Get file names
        f_ref = [file for file in os.listdir(dir_data) if variab in file and model + '_' in file and ref_period in file]
        f_app = [file for file in os.listdir(dir_data) if variab in file and model + '_' in file and dEMT_str in file]
        if len(f_ref)!=1:  sys.exit('File is not unique')
        if len(f_app)!=1:  sys.exit('File is not unique')

        #Read data and calculate difference
        data_ref = xr.open_dataset(dir_data + f_ref[0])
        data_app = xr.open_dataset(dir_data + f_app[0])
        data_diff = data_app.mean('time') - data_ref.mean('time')

        #Save in NetCDF
        fname_tmp = dir_data_Ymax + model + "_tmp.nc"
        data_diff.to_netcdf(fname_tmp)

        #Regrid to standard grid
        fname_grid = dir_regrid + 'grid_xy_1deg'
        fname_regrid = dir_data_Ymax + model + "_regridded.nc"
        os.system("cdo remapbil," + fname_grid + " " + fname_tmp + " " + fname_regrid)

        #Read regridded dataset
        data_regr = xr.open_dataset(fname_regrid)

        #Remove unnecessary variables
        vars_remove = set(data_regr.data_vars).difference([variab])
        data_regr = data_regr.drop(vars_remove)
        if 'height' in data_regr: data_regr = data_regr.drop('height')

        #Collect in one array
        if create==1:
            data_coll = data_regr
            create = 0
        else:
            data_coll = xr.concat((data_coll, data_regr), dim='model')

        #Remove temporary files
        os.remove(fname_tmp)
        os.remove(fname_regrid)

    #Add model names
    data_coll['model'] = all_models        
        
    #Save in file
    fname_out = dir_data_Ymax + variab + '_all_models_yearmax.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
